In [1]:
!pip install --upgrade pandas numpy > /dev/null

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.2 which is incompatible.


In [6]:
import pandas as pd
import numpy as np
print(pd.__version__)
print(np.__version__)

1.5.3
1.22.4


**PANDAS**

In [7]:
%%time
df = pd.read_parquet('yellow_tripdata_2022-01.parquet')

CPU times: user 992 ms, sys: 467 ms, total: 1.46 s
Wall time: 962 ms


In [8]:
df.shape

(2463931, 19)

In [9]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


In [10]:
%%time 
_ = df['passenger_count'].mean()

CPU times: user 16.7 ms, sys: 4.16 ms, total: 20.9 ms
Wall time: 23.7 ms


In [11]:
%%time 
_ = df['passenger_count'].std()

CPU times: user 23.7 ms, sys: 11 ms, total: 34.7 ms
Wall time: 34.9 ms


In [47]:
%%time 
_ = df['passenger_count'].unique()

CPU times: user 1.6 ms, sys: 0 ns, total: 1.6 ms
Wall time: 1.52 ms


In [15]:
len(_)

4

In [16]:
%%time 
_ = df['passenger_count'].cumsum()

CPU times: user 17.6 ms, sys: 0 ns, total: 17.6 ms
Wall time: 59.5 ms


In [18]:
%%time 
_ = df.groupby('VendorID')['passenger_count'].mean()

CPU times: user 55 ms, sys: 5.96 ms, total: 61 ms
Wall time: 90.3 ms


**RAY**

In [19]:
pip install ray

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 104.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.5/468.5 KB 37.7 MB/s eta 0:00:00


In [20]:
import ray
print('Ray Version', ray.__version__)

Ray Version 2.3.0


In [21]:
%%time
ray_dataset = ray.data.read_parquet('yellow_tripdata_2022-01.parquet')

2023-03-21 07:33:27,273	INFO worker.py:1553 -- Started a local Ray instance.
(_get_read_tasks pid=10718) /usr/local/lib/python3.9/dist-packages/ray/data/datasource/parquet_datasource.py:233: FutureWarning: 'ParquetDataset.pieces' attribute is deprecated as of pyarrow 5.0.0 and will be removed in a future version. Use the '.fragments' attribute instead
(_get_read_tasks pid=10718)   pq_ds.pieces, **prefetch_remote_args
(_get_read_tasks pid=10718) /usr/local/lib/python3.9/dist-packages/ray/data/datasource/parquet_datasource.py:311: FutureWarning: 'ParquetDataset.pieces' attribute is deprecated as of pyarrow 5.0.0 and will be removed in a future version. Use the '.fragments' attribute instead
(_get_read_tasks pid=10718)   num_files = len(self._pq_ds.pieces)
(_get_read_tasks pid=10718) /usr/local/lib/python3.9/dist-packages/ray/data/datasource/parquet_datasource.py:324: FutureWarning: 'ParquetDataset.pieces' attribute is deprecated as of pyarrow 5.0.0 and will be removed in a future version

CPU times: user 241 ms, sys: 80 ms, total: 321 ms
Wall time: 5.04 s


Parquet Files Sample: 100%|██████████| 1/1 [00:01<00:00,  1.40s/it]
(_get_read_tasks pid=10718) /usr/local/lib/python3.9/dist-packages/ray/data/datasource/parquet_datasource.py:263: FutureWarning: 'ParquetDataset.pieces' attribute is deprecated as of pyarrow 5.0.0 and will be removed in a future version. Use the '.fragments' attribute instead
(_get_read_tasks pid=10718)   np.array_split(self._pq_ds.pieces, parallelism),


In [22]:
ray_dataset.schema()

VendorID: int64
tpep_pickup_datetime: timestamp[us]
tpep_dropoff_datetime: timestamp[us]
passenger_count: double
trip_distance: double
RatecodeID: double
store_and_fwd_flag: string
PULocationID: int64
DOLocationID: int64
payment_type: int64
fare_amount: double
extra: double
mta_tax: double
tip_amount: double
tolls_amount: double
improvement_surcharge: double
total_amount: double
congestion_surcharge: double
airport_fee: double
-- schema metadata --
pandas: '{"index_columns": [], "column_indexes": [], "columns": [{"name":' + 2492

In [23]:
%%time
from ray.data.aggregate import Max,Mean
ray_dataset.aggregate(Mean("passenger_count"))

2023-03-21 07:36:22,095	INFO bulk_executor.py:39 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[read] -> AllToAllOperator[aggregate]
Shuffle Reduce: 100%|██████████| 1/1 [00:00<00:00, 97.41it/s]

CPU times: user 61.2 ms, sys: 15 ms, total: 76.2 ms
Wall time: 1.61 s


{'mean(passenger_count)': 1.3894533085217193}

In [24]:
%%time
from ray.data.aggregate import Std
ray_dataset.aggregate(Std("passenger_count"))

2023-03-21 07:37:17,682	INFO bulk_executor.py:39 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[read] -> AllToAllOperator[aggregate]
Shuffle Reduce: 100%|██████████| 1/1 [00:00<00:00, 88.08it/s]


CPU times: user 67.1 ms, sys: 7.57 ms, total: 74.6 ms
Wall time: 2 s


{'std(passenger_count)': 0.9829685911276173}

In [26]:
%%time
ray_dataset.groupby('VendorID').mean('passenger_count')

2023-03-21 07:39:01,671	WARNING formatters.py:918 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets>=8` for rich notebook output.


CPU times: user 526 µs, sys: 0 ns, total: 526 µs
Wall time: 520 µs


**DASK**

In [27]:
pip install dask

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [37]:
import dask
import dask.dataframe as dd
print(dask.__version__)

2022.12.1


In [39]:
%%time
df = dd.read_parquet('yellow_tripdata_2022-01.parquet')

CPU times: user 8.41 ms, sys: 2.13 ms, total: 10.5 ms
Wall time: 13.7 ms


In [40]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


In [44]:
%%time
_ = df['passenger_count'].mean().compute()

CPU times: user 36.8 ms, sys: 720 µs, total: 37.5 ms
Wall time: 136 ms


In [45]:
%%time
_ = df['passenger_count'].std().compute()

CPU times: user 49.3 ms, sys: 6.32 ms, total: 55.6 ms
Wall time: 434 ms


In [46]:
%%time
_ = df['passenger_count'].unique()

CPU times: user 2.8 ms, sys: 97 µs, total: 2.9 ms
Wall time: 4.26 ms


In [48]:
%%time
_ = df['passenger_count'].cumsum().compute()

CPU times: user 48.1 ms, sys: 33.7 ms, total: 81.7 ms
Wall time: 368 ms


In [49]:
%%time
_ = df.groupby('VendorID')['passenger_count'].mean().compute()

CPU times: user 90.1 ms, sys: 6.58 ms, total: 96.7 ms
Wall time: 1.03 s


**MODIN**

In [50]:
pip install modin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.8 MB/s eta 0:00:00


In [51]:
import modin
print(modin.__version__)

0.19.0


In [55]:
import os
os.environ["MODIN_ENGINE"] = 'ray'
import modin.pandas as pandas
import ray

In [57]:
%%time
df = pd.read_parquet('yellow_tripdata_2022-01.parquet')

CPU times: user 909 ms, sys: 440 ms, total: 1.35 s
Wall time: 1.35 s


In [58]:
%%time
_ = df['passenger_count'].mean()

CPU times: user 17.3 ms, sys: 3.07 ms, total: 20.4 ms
Wall time: 50.1 ms


In [59]:
%%time
_ = df['passenger_count'].std()

CPU times: user 33.8 ms, sys: 12.2 ms, total: 46 ms
Wall time: 91.8 ms


In [60]:
%%time
_ = df['passenger_count'].unique()

CPU times: user 37.7 ms, sys: 520 µs, total: 38.2 ms
Wall time: 95 ms


In [62]:
%%time
_ = df['passenger_count'].cumsum()

CPU times: user 18 ms, sys: 1.97 ms, total: 20 ms
Wall time: 22.9 ms


In [63]:
%%time
_ = df.groupby('VendorID').mean('passenger_count').mean()

CPU times: user 316 ms, sys: 33.3 ms, total: 349 ms
Wall time: 815 ms


**VAEX**

In [64]:
pip install vaex

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 KB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.2/58.2 KB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.7/238.7 KB 24.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.8/112.8 KB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 KB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 69.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 88.

In [65]:
import vaex
vaex.__version__

{'vaex': '4.16.0',
 'vaex-core': '4.16.1',
 'vaex-viz': '0.5.4',
 'vaex-hdf5': '0.14.1',
 'vaex-server': '0.8.1',
 'vaex-astro': '0.9.3',
 'vaex-jupyter': '0.8.1',
 'vaex-ml': '0.18.1'}

In [68]:
%%time
df = vaex.open('yellow_tripdata_2022-01.parquet')

CPU times: user 4.42 ms, sys: 13 µs, total: 4.43 ms
Wall time: 6.5 ms


In [69]:
%%time
_ = df['passenger_count'].mean()

CPU times: user 138 ms, sys: 39.5 ms, total: 178 ms
Wall time: 283 ms


In [70]:
%%time
_ = df['passenger_count'].std()

CPU times: user 503 ms, sys: 73.5 ms, total: 576 ms
Wall time: 768 ms


In [71]:
%%time
_ = df['passenger_count'].unique()

CPU times: user 94.5 ms, sys: 48.1 ms, total: 143 ms
Wall time: 162 ms


In [81]:
%%time
_ = df.groupby(df.VendorID, agg={'p_mean': [vaex.agg.mean('passenger_count')]})

CPU times: user 408 ms, sys: 74.6 ms, total: 482 ms
Wall time: 972 ms
